In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [3]:
from ease_recommender import *
from npmi_recommender import *

import pickle as p

In [4]:
print("loading cache data...")
D = p.load(open("cached_data/movie_lens_preprocessed.p", "rb"))

row = D["userId"]
col = D["movieId"]
data = D["rating"]

movies = D["movies"]

print("done")

loading cache data...
done


In [5]:
def find_match_using_terms(terms, movies=movies, case_insensitive=False):
    assert type(terms) in (list, tuple, set)
    
    title = movies.title
    if case_insensitive:
        title = title.str.lower()
    
    matches = True
    for term in terms:
        matches &= title.str.contains(term)
        
    matches = np.where(matches)[0]

    if len(matches) > 1:
        raise Exception("Multiple matches found, filter down to a single match", matches)
        
    return matches[0]

In [6]:
mat = csr_matrix((data.astype(bool), (row, col))).astype(np.int64)

In [7]:
a = find_match_using_terms(["Sense and Sensibility", "1995"])

movies.loc[a]

title         Sense and Sensibility (1995)
genres                       Drama|Romance
imdbId                              114388
tmdbId                              4584.0
avg_rating                        7.891793
num_votes                             8811
Name: 16, dtype: object

In [8]:
b = find_match_using_terms(["Pride and Prejudice", "1995"])
# b = find_match_using_terms(["Amadeus"])

movies.loc[b]

title         Pride and Prejudice (1995)
genres                     Drama|Romance
imdbId                            112130
tmdbId                          164721.0
avg_rating                       7.98947
num_votes                           1517
Name: 7228, dtype: object

In [9]:
lambda_ = optimize_lambda_using_a_to_b_matching_npmi(mat, a, b)

lambda_: 0.3819660112501051
error: 29.381966011250107
lambda_: 0.6180339887498948
error: 42.61803398874989
lambda_: 0.2360679774997897
error: 23.23606797749979
lambda_: 0.14589803375031546
error: 23.145898033750317
lambda_: 0.18811285114599988
error: 23.188112851146
lambda_: 0.09016994374947425
error: 27.090169943749473
lambda_: 0.12461179749810727
error: 24.12461179749811
lambda_: 0.16202265916659658
error: 22.162022659166595
lambda_: 0.16679509516180432
error: 22.166795095161806
lambda_: 0.16424039852041972
error: 22.16424039852042
lambda_: 0.15586360031343763
error: 22.155863600313438
lambda_: 0.15205709260347444
error: 22.152057092603474
lambda_: 0.14970454146027865
error: 23.149704541460277
lambda_: 0.15351104917024186
error: 22.15351104917024
lambda_: 0.15115849802704606
error: 22.151158498027048
lambda_: 0.15060313603670702
error: 22.150603136036707
lambda_: 0.1502599034506177
error: 22.150259903450618
lambda_: 0.150047774046368
error: 22.150047774046367
lambda_: 0.1499166708645

In [10]:
# a_to_b_error_metric_npmi_pop_weighted(mat, a, b, 0, lambda_penalty=False)

In [11]:
a_to_b_error_metric_npmi_pop_weighted(mat, a, b, lambda_, lambda_penalty=False)

lambda_: 0.1499206414373801
error: 11.0


11.0

In [12]:
top_k = 20

In [13]:
# using normalized pointwise mutual information (popularity weighted)

similarity_scores = npmi_batch_popularity_weighted(mat, a, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
815,Emma (1996),Comedy|Drama|Romance,116191,3573.0,7.485110,2392
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
492,Much Ado About Nothing (1993),Comedy|Romance,107616,11971.0,7.734885,4635
510,"Remains of the Day, The (1993)",Drama|Romance,107943,1245.0,7.783682,3555
258,Little Women (1994),Drama,110367,9587.0,7.199093,2599
262,Like Water for Chocolate (Como agua para choco...,Drama|Fantasy|Romance,103994,18183.0,7.827167,4205
352,Four Weddings and a Funeral (1994),Comedy|Romance,109831,712.0,7.284194,6370
57,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance,110877,11010.0,7.927828,5037
504,"Piano, The (1993)",Drama|Romance,107822,713.0,7.368306,5160


In [14]:
# using normalized pointwise mutual information (popularity weighted)

similarity_scores = npmi_batch_popularity_weighted(mat, b, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
15953,North & South (2004),Drama|Romance,417349,147269.0,8.057732,243
9966,Pride & Prejudice (2005),Drama|Romance,414387,4348.0,7.703508,3062
13373,Persuasion (2007),Drama|Romance,844330,13949.0,7.719715,150
15308,Northanger Abbey (2007),Drama|Romance,844794,18093.0,7.429658,84
12497,"Young Victoria, The (2009)",Drama|Romance,962736,18320.0,7.472767,247
14718,Jane Eyre (2011),Drama|Romance,1229822,38684.0,7.451439,288
16,Sense and Sensibility (1995),Drama|Romance,114388,4584.0,7.891793,8811
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
10919,Becoming Jane (2007),Drama|Romance,416508,2977.0,7.129006,273
